In [20]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import openpyxl

# Load the dataset
df = pd.read_excel('Online Retail.xlsx')

# Filter out products bought more than 20 times
product_counts = df['Description'].value_counts()
frequent_products = product_counts[product_counts > 20]
df_filtered = df[df['Description'].isin(frequent_products.index)]

# Create a basket matrix
basket = df_filtered.pivot_table(index='InvoiceNo', columns='Description', values='Quantity', aggfunc='sum', fill_value=0)

# Convert to binary values: 1 if product was purchased (Quantity > 0), 0 otherwise
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Apply Apriori algorithm with a minimum support of 0.01 and a minimum lift of 1.2
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True, low_memory=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.2)

# Function to recommend products based on a given product description
def recommend_products(product_description, rules, top_n=5):
    # Filter rules that have the given product in the antecedent
    filtered_rules = rules[rules['antecedents'].apply(lambda x: product_description in x)]
    
    # Sort the filtered rules by lift in descending order
    sorted_rules = filtered_rules.sort_values(by='lift', ascending=False)
    
    # Get the top N recommended products
    recommendations = set()  # Use a set to automatically handle duplicates
    
    for _, rule in sorted_rules.iterrows():
        consequent = tuple(rule['consequents'])
        recommendations.update(consequent)  # Add consequent products to the set
        
    # Remove the original product from the recommendations (if present)
    recommendations.discard(product_description)
    
    # Limit to top N recommendations
    return list(recommendations)[:top_n]

# Example usage: Recommend products for a given product
product_description = 'WHITE HANGING HEART T-LIGHT HOLDER'  # Example product
recommended_products = recommend_products(product_description, rules, top_n=5)

# Display the recommended products
print(f"Recommendations for '{product_description}':")
for product in recommended_products:
    print(f"- {product}")


C:\Users\Sher\AppData\Local\Temp\ipykernel_21492\2915185881.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)
c:\Users\Sher\miniconda3\envs\pycaret3.11\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Recommendations for 'WHITE HANGING HEART T-LIGHT HOLDER':
- JAM MAKING SET WITH JARS
- REGENCY CAKESTAND 3 TIER
- RED HANGING HEART T-LIGHT HOLDER
- CANDLEHOLDER PINK HANGING HEART
- HEART OF WICKER SMALL
